In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine 
import os
import re

In [ ]:
df = pd.read_csv('parsed_issues.csv', encoding = 'cp1252', na_filter = False)
df.insert(2, 'tableId_rowIndex', '')
df.insert(3, 'rowCounter', '')

In [ ]:
df['tableId_rowIndex'] = df['tableId'] + df['rowIndex'].astype(str)

In [ ]:
df['rowCounter'] = df.groupby('tableId_rowIndex').cumcount()+1

In [ ]:
df = df.drop(['tableId_rowIndex'], axis = 1)
print(len(df))

In [ ]:
df = df.replace(np.nan, '', regex=True)

In [ ]:
df.to_csv('issue_parsed_clean.csv', index = False,encoding = 'utf-8-sig')

Preparing Test Data for Running BERT Model

In [ ]:
# Importing environmental variables library that reads from the .env file
from dotenv import load_dotenv

# Loading key-value pairs from the .env file into the OS environment
load_dotenv()

# Reading the key-value pairs from the OS environment
user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
db_hostname = os.getenv("DB_HOST")
db_name = os.getenv("DB_DATABASE")

# Using those variables in the connection string using "F" strings
conn_string = f"mysql+mysqldb://{user}:{password}@{db_hostname}/{db_name}?charset=utf8mb4"
engine = create_engine(conn_string)

In [ ]:
query = "SELECT tableId, rowIndex, rowCounter, issue_parsed FROM pcmr.issues_parsed;"
with engine.connect() as conn:
    df = pd.read_sql(query, conn)
df.head(5)

In [ ]:
df.rename(columns={'issue_parsed':'status_txt'}, inplace=True)
df.head(5)

In [ ]:
test_df = pd.read_csv(r'C:\Users\t1nipun\Desktop\BERT_workshop_full\BERT_workshop\demonstrate\data\test.csv', encoding ='utf-8-sig')

In [ ]:
test_data = pd.concat([test_df, df], sort=False)

In [ ]:
test_data.to_csv(r'C:\Users\t1nipun\Desktop\BERT_workshop_full\BERT_workshop\demonstrate\data\test.csv', index = False,encoding = 'utf-8-sig')

Prepaing Data to populate to SQL Database

In [ ]:
test_df_statusLabels = pd.read_csv(r'C:\Users\t1nipun\Desktop\BERT_workshop_full\BERT_workshop\demonstrate\test_data_BERT_results.csv', encoding ='utf-8-sig')
test_df_statusLabels

In [ ]:
test_data['status'] = test_df_statusLabels['pred_status_label']

In [ ]:
print(len(test_data))

In [ ]:
train_data = pd.read_csv(r'C:\Users\t1nipun\Desktop\BERT_workshop_full\BERT_workshop\demonstrate\data\train.csv', encoding ='utf-8-sig')
print(len(train_data))

In [ ]:
val_data = pd.read_csv(r'C:\Users\t1nipun\Desktop\BERT_workshop_full\BERT_workshop\demonstrate\data\dev.csv', encoding ='utf-8-sig')
print(len(val_data))

In [ ]:
final_data = pd.concat([test_data, train_data, val_data], sort=False)
print(len(final_data))

In [ ]:
final_data.to_csv('status_labels.csv', index = False,encoding = 'utf-8-sig')

Populating Status Labels on Database